# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
mongo.list_database_names()

['admin', 'classdb', 'config', 'local', 'met', 'uk_food']

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [23]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [41]:
# review the collections in our new database
db.establishments.find_one({})

{'_id': ObjectId('64b445aef2207c23079d6810'),
 'FHRSID': 254719,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DY',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2022-03-24T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'},
 'RightToReply': '',
 'Distance': 4587.347174863443,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  'returncode

In [58]:
# review a document in the establishments collection
establishments = db['establishments']

In [59]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [74]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [88]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)


In [101]:
# Check that the new restaurant was inserted
insert_result.inserted_id

ObjectId('64b4460e4ce411600db919e5')

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [113]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
results = establishments.find_one({"BusinessTypeID": "Restaurant/Cafe/Canteen",
                                   "BusinessTypeID": {"$ne": ""}}, # our new restaurant does not have a value for this field
                                  ["BusinessTypeID", "BusinessType"])
results

{'_id': ObjectId('64b445aef2207c23079d6810'),
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [124]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({"BusinessName":"Penang Flavours"},
                          [{"$set": {"BusinessTypeID": 1}}])

In [134]:
# Confirm that the new restaurant was updated
establishments.find_one({"BusinessName":"Penang Flavours"}, ["BusinessName", "BusinessTypeID"])

{'_id': ObjectId('64b4460e4ce411600db919e5'),
 'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [143]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_query = {"LocalAuthorityName": "Dover"}
establishments.count_documents(dover_query)

994

In [151]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(dover_query)

In [158]:
# Check if any remaining documents include Dover
establishments.count_documents(dover_query)

0

In [164]:
# Check that other documents remain with 'find_one'
establishments.find_one({}, ["BusinessName", "LocalAuthorityName"])

{'_id': ObjectId('64b445aef2207c23079d6aee'),
 'BusinessName': 'The Pavilion',
 'LocalAuthorityName': 'Folkestone and Hythe'}

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [169]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{'$set': {"geocode.latitude": {'$toDouble': "$geocode.latitude"},
                                          "geocode.longitude": {'$toDouble': "$geocode.longitude"}
                                          } 
                                 }]
                           )

Use `update_many` to convert `RatingValue` to integer numbers.

In [170]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [173]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}])

In [175]:
# Check that the coordinates and rating value are now numbers
establishments.find_one({},
                        ["BusinessName", "geocode.latitude", "geocode.longitude", "RatingValue"])

{'_id': ObjectId('64b445aef2207c23079d6aee'),
 'BusinessName': 'The Pavilion',
 'RatingValue': 5,
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812}}